<a href="https://colab.research.google.com/github/gspracklin/genomics_scripts/blob/master/eigen_byarm_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Eigenvector Analysis by chrom arm

###Install requirements

In [0]:
!pip install cooler
!pip install bioframe
!pip install cython
!pip install git+https://github.com/mirnylab/cooltools

    100% |████████████████████████████████| 2.1MB 10.9MB/s 
  Cloning https://github.com/mirnylab/cooltools to /tmp/pip-req-build-ew9d1lxa
    100% |████████████████████████████████| 686kB 18.6MB/s 
    100% |████████████████████████████████| 3.2MB 9.0MB/s 
    100% |████████████████████████████████| 3.8MB 10.6MB/s 
    100% |████████████████████████████████| 16.1MB 2.1MB/s 
    100% |████████████████████████████████| 163kB 24.2MB/s 
  Running setup.py bdist_wheel for cooltools ... - \ | / - \ | done
  Stored in directory: /tmp/pip-ephem-wheel-cache-89z1dn1v/wheels/bf/59/ac/f23113200c2678fc1dd5901a905c180fc6392ae3af501292d0
Successfully built cooltools


In [0]:
!wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64.v369/bedGraphToBigWig
!chmod 755 bedGraphToBigWig

--2018-12-11 19:45:38--  http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64.v369/bedGraphToBigWig
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4380534 (4.2M) [text/plain]
Saving to: ‘bedGraphToBigWig.2’

bedGraphToBigWig.2  100%[===================>]   4.18M  3.22MB/s    in 1.3s    

2018-12-11 19:45:39 (3.22 MB/s) - ‘bedGraphToBigWig.2’ saved [4380534/4380534]



In [0]:
cp bedGraphToBigWig /usr/bin

In [0]:
#Note: !bedGraphToBigWig needs to be executable (i.e. not !./bedGraphToBigWig)
!bedGraphToBigWig 

bedGraphToBigWig v 4 - Convert a bedGraph file to bigWig format.
usage:
   bedGraphToBigWig in.bedGraph chrom.sizes out.bw
where in.bedGraph is a four column file in the format:
      <chrom> <start> <end> <value>
and chrom.sizes is a two-column file/URL: <chromosome name> <size in bases>
and out.bw is the output indexed big wig file.
If the assembly <db> is hosted by UCSC, chrom.sizes can be a URL like
  http://hgdownload.cse.ucsc.edu/goldenPath/<db>/bigZips/<db>.chrom.sizes
or you may use the script fetchChromSizes to download the chrom.sizes file.
If not hosted by UCSC, a chrom.sizes file can be generated by running
twoBitInfo on the assembly .2bit file.
The input bedGraph file must be sorted, use the unix sort command:
  sort -k1,1 -k2,2n unsorted.bedGraph > sorted.bedGraph
options:
   -blockSize=N - Number of items to bundle in r-tree.  Default 256
   -itemsPerSlot=N - Number of data points bundled at lowest level. Default 1024
   -unc - If set, do not use compression.


###Upload .cool  Files:

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

### chromarm eigenvector

Steps:

* Download `twoBitToFa` and the hg19 sequence
* Convert to fasta
* Download hg19 gapfile
* Generate chromarm table to get the `regions`
* Get the bin table from the cooler
* Load the fasta file and calculate GC content on your bin table
* Pass the bin table with GC and the list of `regions` to `cooltools.eigdecomp.cooler_cis_eig`

In [0]:
!wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64.v369/twoBitToFa
!chmod 755 twoBitToFa

--2018-12-11 19:45:42--  http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64.v369/twoBitToFa
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4716626 (4.5M) [text/plain]
Saving to: ‘twoBitToFa’

twoBitToFa          100%[===================>]   4.50M  3.58MB/s    in 1.3s    

2018-12-11 19:45:43 (3.58 MB/s) - ‘twoBitToFa’ saved [4716626/4716626]



In [0]:
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/bigZips/hg19.2bit

--2018-12-11 19:45:46--  http://hgdownload.cse.ucsc.edu/goldenPath/hg19/bigZips/hg19.2bit
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 816241703 (778M) [text/plain]
Saving to: ‘hg19.2bit’

hg19.2bit           100%[===================>] 778.43M  30.9MB/s    in 16s     

2018-12-11 19:46:03 (47.7 MB/s) - ‘hg19.2bit’ saved [816241703/816241703]



In [0]:
!./twoBitToFa hg19.2bit hg19.fa

In [0]:
import bioframe
import cooltools.eigdecomp

In [0]:
hg19 = bioframe.fetch_chromsizes('hg19')
gaps = bioframe.read_gapfile('http://hgdownload.cse.ucsc.edu/goldenPath/hg19/database/gap.txt.gz')
gaps = gaps[gaps['type'] == 'centromere']
gaps['mid'] = (gaps['start'] + gaps['end']) // 2
gaps.head()

,chrom,start,end,length,type,bridge,mid
15,chr1,121535434,124535434,3000000,centromere,no,123035434
46,chr10,39254935,42254935,3000000,centromere,no,40754935
68,chr11,51644205,54644205,3000000,centromere,no,53144205
80,chr12,34856694,37856694,3000000,centromere,no,36356694
87,chr13,16000000,19000000,3000000,centromere,no,17500000


In [0]:
chromarms = bioframe.tools.split_chromosomes(hg19, gaps.set_index('chrom')['mid'], suffixes=('p', 'q'))
chromarms.head()

,chrom,start,end,name
0,chr1,0,123035434,chr1p
1,chr1,123035434,249250621,chr1q
2,chr2,0,93826171,chr2p
3,chr2,93826171,243199373,chr2q
4,chr3,0,92004854,chr3p


In [0]:
import cooler
clr = cooler.Cooler('DKO.20000.cool')
clr

<Cooler "DKO.20000.cool::/">

In [0]:
fasta_records = bioframe.load_fasta('hg19.fa')

In [0]:
bins = clr.bins()[:]
fasta_records = bioframe.load_fasta('hg19.fa')
bins['GC'] = bioframe.tools.frac_gc(bins, fasta_records)
bins.head()

,chrom,start,end,weight,GC
0,chr1,0,20000,NaN,0.59230
1,chr1,20000,40000,NaN,0.49430
2,chr1,40000,60000,NaN,0.36075
3,chr1,60000,80000,NaN,0.35660
4,chr1,80000,100000,NaN,0.40255


In [0]:
lam, vec = cooltools.eigdecomp.cooler_cis_eig(
    clr, 
    bins, 
    regions=[x[:3] for x in chromarms.itertuples(index=False)], 
    n_eigs=3, 
    phasing_track_col='GC', 
    balance='weight', 
    ignore_diags=None, 
    clip_percentile=99.9, 
    sort_metric=None
)

In [0]:
vec.head()

,chrom,start,end,weight,GC,E1,E2,E3
0,chr1,0,20000,NaN,0.59230,NaN,NaN,NaN
1,chr1,20000,40000,NaN,0.49430,NaN,NaN,NaN
2,chr1,40000,60000,NaN,0.36075,NaN,NaN,NaN
3,chr1,60000,80000,NaN,0.35660,NaN,NaN,NaN
4,chr1,80000,100000,NaN,0.40255,NaN,NaN,NaN


In [0]:
lam.head()

,eigval1,eigval2,eigval3
region,,,
chr1:0-123035434,1715.509511,1119.805776,-890.544007
chr1:123035434-249250621,1142.952018,1026.974430,-868.326597
chr2:0-93826171,1382.297385,770.204333,-690.263584
chr2:93826171-243199373,1845.107407,1106.526004,-1104.508453
chr3:0-92004854,1392.531753,807.529687,-800.195451


### Write and download file to local drive

In [0]:
#write dataframe to file
bioframe.to_bigwig(vec, hg19, 'eigen_byarm.bw', 'E1')

bedGraphToBigWig /tmp/tmpsprvbqnp.bg /tmp/tmpo0112kb7.chrom.sizes eigen_byarm.bw


In [0]:
from google.colab import files

files.download('eigen_byarm.bw')